In [37]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import tensorflow as tf

In [38]:
df = pd.read_csv(r"C:\Users\jariv\Project4\online_retail.csv")
df.head()

C:\Users\jariv\AppData\Local\Temp\ipykernel_6372\3201789302.py:1: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850.0,United Kingdom
1,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850.0,United Kingdom
2,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
3,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [40]:
# Droping rows having missing values
df = df.dropna()
df.shape

(406829, 8)

In [41]:
# Drop rows with "POST" in the StockCode column
df = df[df['StockCode'] != 'POST']

# Drop rows where InvoiceNo column starts with "C"
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [42]:
#add new column Total Price
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850.0,United Kingdom,25.50
1,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850.0,United Kingdom,15.30
2,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom,20.34
3,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom,20.34


In [43]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.info()

C:\Users\jariv\AppData\Local\Temp\ipykernel_6372\40610843.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



<class 'pandas.core.frame.DataFrame'>
Index: 396825 entries, 0 to 532617
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    396825 non-null  object        
 1   StockCode    396825 non-null  object        
 2   Description  396825 non-null  object        
 3   Quantity     396825 non-null  int64         
 4   InvoiceDate  396825 non-null  datetime64[ns]
 5   UnitPrice    396825 non-null  float64       
 6   CustomerID   396825 non-null  float64       
 7   Country      396825 non-null  object        
 8   TotalPrice   396825 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.3+ MB


In [44]:
#update dtypes
df['CustomerID'] = df['CustomerID'].astype(int)
df['InvoiceNo'] = df['InvoiceNo'].astype(int)
df['StockCode'] = df['StockCode'].astype(str)
df['Description'] = df['Description'].astype(str)
df['Country'] = df['Country'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396825 entries, 0 to 532617
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    396825 non-null  int32         
 1   StockCode    396825 non-null  object        
 2   Description  396825 non-null  object        
 3   Quantity     396825 non-null  int64         
 4   InvoiceDate  396825 non-null  datetime64[ns]
 5   UnitPrice    396825 non-null  float64       
 6   CustomerID   396825 non-null  int32         
 7   Country      396825 non-null  object        
 8   TotalPrice   396825 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(3)
memory usage: 27.2+ MB


In [45]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [46]:
df['Recency'] = (df['InvoiceDate'].max() - df['InvoiceDate'])
df['Recency'] = df['Recency'].dt.days
df.tail(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Recency
532608,581587,22629,SPACEBOY LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680,France,23.4,0
532609,581587,22631,CIRCUS PARADE LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680,France,23.4,0
532610,581587,22726,ALARM CLOCK BAKELIKE GREEN,4,2011-12-09 12:50:00,3.75,12680,France,15.0,0
532611,581587,22727,ALARM CLOCK BAKELIKE RED,4,2011-12-09 12:50:00,3.75,12680,France,15.0,0
532612,581587,22728,ALARM CLOCK BAKELIKE PINK,4,2011-12-09 12:50:00,3.75,12680,France,15.0,0
532613,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680,France,15.0,0
532614,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.6,0
532615,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.6,0
532616,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.6,0
532617,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09 12:50:00,4.15,12680,France,16.6,0


In [47]:
#Frequency
frequency_df = df.groupby('CustomerID')['InvoiceDate'].count().reset_index()
frequency_df.rename(columns={'InvoiceDate': 'Frequency'}, inplace=True)
df = df.merge(frequency_df, on='CustomerID', how='left', suffixes=('_original', '_frequency'))
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Recency,Frequency
396820,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680,France,15.0,0,49
396821,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.6,0,49
396822,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.6,0,49
396823,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.6,0,49
396824,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09 12:50:00,4.15,12680,France,16.6,0,49


In [48]:
df = df.groupby('CustomerID').agg({
    'Frequency': 'first',
    'TotalPrice': 'sum',
    'UnitPrice': 'mean',
    'Quantity': 'mean',
    'Recency': 'first'
}).reset_index()

df = df.rename({'TotalPrice': 'MonetaryValue'}, axis='columns')

df.head()

,CustomerID,Frequency,MonetaryValue,UnitPrice,Quantity,Recency
0,12346,1,77183.60,1.040000,74215.000000,325
1,12347,182,4310.00,2.644011,13.505495,366
2,12348,27,1437.24,0.692963,86.370370,357
3,12349,72,1457.55,4.237500,8.750000,18
4,12350,16,294.40,1.581250,12.250000,309


In [49]:
recency_scores = [5, 4, 3, 2, 1] #higher score = (more recent)
frequency_scores = [1, 2, 3, 4, 5] #higher score = (higher frequency)
monetary_scores = [1, 2, 3, 4, 5] # higher score = higher monetary value

#calculate scores
df['RecencyScore'] = pd.cut(df['Recency'], bins=5, labels=recency_scores)
df['FrequencyScore'] = pd.cut(df['Frequency'], bins=5, labels=frequency_scores)
df['MonetaryScore'] = pd.cut(df['MonetaryValue'], bins=5, labels=monetary_scores)
df.head()

,CustomerID,Frequency,MonetaryValue,UnitPrice,Quantity,Recency,RecencyScore,FrequencyScore,MonetaryScore
0,12346,1,77183.60,1.040000,74215.000000,325,1,1,2
1,12347,182,4310.00,2.644011,13.505495,366,1,1,1
2,12348,27,1437.24,0.692963,86.370370,357,1,1,1
3,12349,72,1457.55,4.237500,8.750000,18,5,1,1
4,12350,16,294.40,1.581250,12.250000,309,1,1,1


In [50]:
df['RecencyScore'] = df['RecencyScore'].astype(int)
df['FrequencyScore'] = df['FrequencyScore'].astype(int)
df['MonetaryScore'] = df['MonetaryScore'].astype(int)

In [51]:
#calculate RFM score by combining the three scores
df['RFM_Score'] = df['RecencyScore'] + df['FrequencyScore'] + df['MonetaryScore']

segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
df['Value Segment'] = pd.qcut(df['RFM_Score'], q=3, labels=segment_labels)
df.head()

,CustomerID,Frequency,MonetaryValue,UnitPrice,Quantity,Recency,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Value Segment
0,12346,1,77183.60,1.040000,74215.000000,325,1,1,2,4,Low-Value
1,12347,182,4310.00,2.644011,13.505495,366,1,1,1,3,Low-Value
2,12348,27,1437.24,0.692963,86.370370,357,1,1,1,3,Low-Value
3,12349,72,1457.55,4.237500,8.750000,18,5,1,1,7,High-Value
4,12350,16,294.40,1.581250,12.250000,309,1,1,1,3,Low-Value


In [52]:
# Define the order of x-axis categories
category_order = ['High-Value', 'Mid-Value']

# Create the bar chart with specified x-axis category order
fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count',
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution',
                          category_orders={'Value Segment': category_order})
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)
fig_segment_dist.show()

c:\Users\jariv\anaconda3\Lib\site-packages\plotly\express\_core.py:1958: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [66]:
df['RFM Customer Segments'] = ''

#assign RFM segments based on RFM score
df.loc[df['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
df.loc[(df['RFM_Score'] >= 6) & (df['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Loyal Customers'
df.loc[(df['RFM_Score'] >= 5) & (df['RFM_Score'] < 6), 'RFM Customer Segments'] = 'Average'
df.loc[(df['RFM_Score'] >= 4) & (df['RFM_Score'] < 5), 'RFM Customer Segments'] = "At Risk"
df.loc[(df['RFM_Score'] >= 3) & (df['RFM_Score'] < 4), 'RFM Customer Segments'] = "Gone"

print(df[['CustomerID', 'RFM Customer Segments']])

      CustomerID RFM Customer Segments
0          12346               At Risk
1          12347                  Gone
2          12348                  Gone
3          12349       Loyal Customers
4          12350                  Gone
...          ...                   ...
4334       18280               At Risk
4335       18281               Average
4336       18282       Loyal Customers
4337       18283                  Gone
4338       18287               Average

[4339 rows x 2 columns]


In [67]:
segment_product_counts = df.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['Value Segment', 'RFM Customer Segments'],
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

C:\Users\jariv\AppData\Local\Temp\ipykernel_6372\3596661949.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\jariv\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\jariv\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [63]:
# Define the desired order of 'Value Segment'
value_segment_order = ['High-Value', 'Mid-Value', 'Low-Value']

# Convert 'Value Segment' to a categorical variable with the specified order
segment_product_counts['Value Segment'] = pd.Categorical(segment_product_counts['Value Segment'], categories=value_segment_order, ordered=True)

# Sort the DataFrame based on the new order of 'Value Segment' and the count
segment_product_counts['Value Segment'] = segment_product_counts['Value Segment'].cat.reorder_categories(value_segment_order, ordered=True)
segment_product_counts = segment_product_counts.sort_values(['Value Segment', 'Count'], ascending=[True, False])
# Create the Treemap plot with the updated order of 'Value Segment'
fig_treemap_segment_product = px.treemap(segment_product_counts, 
                                         path=['Value Segment', 'RFM Customer Segments'], 
                                         values='Count', 
                                         color='Value Segment', 
                                         color_discrete_sequence=px.colors.qualitative.Pastel, 
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()


c:\Users\jariv\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\jariv\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [68]:
# Filter the data to include only the customers in the Champions segment
loyal_customer_segment = df[df['RFM Customer Segments'] == 'Loyal Customers']

fig = go.Figure()
fig.add_trace(go.Box(y=loyal_customer_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=loyal_customer_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=loyal_customer_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

fig.show()

In [69]:
correlation_matrix = loyal_customer_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

# Visualize the correlation matrix using a heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

In [70]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = df['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
loyalists_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[loyalists_color if segment == 'Potenial Loyalists' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

In [72]:
# Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = df.groupby('RFM Customer Segments')[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].mean().reset_index()

# Create a grouped bar chart to compare segment scores
fig = go.Figure()

# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

# Update the layout
fig.update_layout(
    title='Comparison of RFM Segments based on AVG Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()

In [73]:
df.head()

,CustomerID,Frequency,MonetaryValue,UnitPrice,Quantity,Recency,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Value Segment,RFM Customer Segments
0,12346,1,77183.60,1.040000,74215.000000,325,1,1,2,4,Low-Value,At Risk
1,12347,182,4310.00,2.644011,13.505495,366,1,1,1,3,Low-Value,Gone
2,12348,27,1437.24,0.692963,86.370370,357,1,1,1,3,Low-Value,Gone
3,12349,72,1457.55,4.237500,8.750000,18,5,1,1,7,High-Value,Loyal Customers
4,12350,16,294.40,1.581250,12.250000,309,1,1,1,3,Low-Value,Gone


In [74]:
df.to_csv('RFM.csv',index=False)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the parameter distributions to sample from
param_dist = {'n_neighbors': randint(1, 10), 'weights': ['uniform', 'distance']}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Instantiate RandomizedSearchCV
random_search = RandomizedSearchCV(knn, param_distributions=param_dist, n_iter=5, cv=5)

# Fit the RandomizedSearchCV object to find the best hyperparameters
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = random_search.best_estimator_

# Make predictions using the best model
y_pred = best_model.predict(X_test)